## Долгосрочная работа по алгебре на тему "Введение в базисы Гребнера" срок сдачи 22.12.22

Цель данной работы -- написание собственного алгоритма для вычисления редуцированного базиса Гребнера полиномиального идеала, а также вспомогательных функций. В процессе работы будут использоваться основные функции системы КА Sage для работы с многочленами от нескольких переменных.

В справочной информации системы КА Sage доступной [ссылке](https://doc.sagemath.org/html/en/reference/polynomial_rings/sage/rings/polynomial/multi_polynomial_ideal.html). Приводятся примеры работы с многочленами от нескольких переменных, а также примеры вычисления базисов Гребнера. Отметим, что используемые в данных примерах функции для вычисления базисов Гребнера сразу вычисляют редуцированный базис Гребнера. Однако для целей освоения материала будет правильнее самостоятельно написать все необходимые вспомогательные процедуры.

### Мономиальные порядки

В Sage можно несколькими способами создавать кольцо многочленов от нескольких переменных. Мы будем использовать следующий наиболее подробный способ.

In [37]:
R.<x,y> = PolynomialRing(QQ, order='lex')
R

Multivariate Polynomial Ring in x, y over Rational Field

Посмотреть порядок переменных можно командой

In [38]:
R.term_order(), R.gens()

(Lexicographic term order, (x, y))

В случае `lex` порядка имеем

In [39]:
x > y^100

True

Таким образом относительно данного порядка многочлен $y^{100} + x^2 + xy$ запишется в виде

In [40]:
R(y^100 + x^2 + x*y)

x^2 + x*y + y^100

Заметим, что если мы передадим перемнные в другом порядке, то используется другой `lex` порядок

In [42]:
R.<y,x> = PolynomialRing(QQ, order='lex')
x > y

False

И в данном случае многочлен $y^{100} + x^2 + xy$ запишется в виде

In [43]:
R(y^100 + x^2 + x*y)

y^100 + y*x + x^2

Используя справочную информацию, а именно, сочетание клавиш `Shift+Tab` перечислите названия мономиальных порядков из Sage соответствующие мономиальным порядкам `grlex`, `grevlex`. Отметим, что для смены мономиального порядка можно использовать встроенную функцию `.change_ring()`

In [ ]:
## your code here

создайте кольцо многочленов от двух переменных с мономиальными порядками `grlex` и `grevlex` относительно переменных $(x,y)$ и $(y,x)$ и запишите многочлен $y^{100} + yx + x^2$ в порядке убывания мономов

In [46]:
## your code here

### Алгоритм деления

Далее напишем алгоритм деления многочленов от нескольких переменных с остатком. Нам понадобятся встроенные функции вычисления старшего члена и старшего монома. Приведем примеры

In [127]:
R.<x,y> = PolynomialRing(QQ, order='lex')
f = 2*x^5 + y^6 + x^2*y + 12
f

2*x^5 + x^2*y + y^6 + 12

In [128]:
print(f'LT(f)={f.lt()},  LM(f)={f.lm()}, LC(f)={f.lc()}')

LT(f)=2*x^5,  LM(f)=x^5, LC(f)=2


Обратите внимание, что старший моном и многочлен имеют одинаковый тип

In [56]:
type(f), type(f.lt())

(<class 'sage.rings.polynomial.multi_polynomial_libsingular.MPolynomial_libsingular'>,
 <class 'sage.rings.polynomial.multi_polynomial_libsingular.MPolynomial_libsingular'>)

Однако при делении многочленов тип меняется. Рассмотрим пример

In [62]:
R.<x,y, z> = PolynomialRing(QQ, order='lex')
f = x^3
g = x^2
h = f / g
print(h)
type(f), type(h)

x


(<class 'sage.rings.polynomial.multi_polynomial_libsingular.MPolynomial_libsingular'>,
 <class 'sage.rings.fraction_field_element.FractionFieldElement'>)

Поэтому полсле деления будем приводить тип

In [64]:
R.<x,y, z> = PolynomialRing(QQ, order='lex')
f = x^3
g = x^2
h = R(f / g)
print(h)
type(f), type(h)

x


(<class 'sage.rings.polynomial.multi_polynomial_libsingular.MPolynomial_libsingular'>,
 <class 'sage.rings.polynomial.multi_polynomial_libsingular.MPolynomial_libsingular'>)

Однако можно было бы использовать и деление нацело `//`

In [110]:
R.<x,y, z> = PolynomialRing(QQ, order='lex')
f = x^3
g = x^2
h = f // g
print(h)
type(f), type(h)

x


(<class 'sage.rings.polynomial.multi_polynomial_libsingular.MPolynomial_libsingular'>,
 <class 'sage.rings.polynomial.multi_polynomial_libsingular.MPolynomial_libsingular'>)

Далее реализуем базовый алгоритм деления

In [100]:
def divide_with_rem(g, F, verbose=False):
    R = g.parent()
    
    q = [R.zero() for _ in F] # quotients
    rem = R.zero() # remainder
    r = g # after finish this polynomial will be zero
    
    assert g == r + sum(fi*qi for fi,qi in zip(F, q)) + rem 
    
    step = 0
    if verbose:
        print(f'after step {step} r={r},  q={q}, rem={rem}:')
    while not r.is_zero():
        was_div = False
        for i, fi in enumerate(F):
            if r.lm() % fi.lm() == 0:
                mult = R(r.lt() / fi.lt())
                q[i] += # your code here
                r = r - fi * mult
                was_div = True
                break
        if not was_div:
            rem = rem + r.lt()
            r = r - # your code here
        
        assert g == r + sum(fi*qi for fi,qi in zip(F, q)) + rem 
        step += 1
        if verbose:
            print(f'after step {step} r={r},  q={q}, rem={rem}:')
        
    return q, rem 

Проверим наш алгоритм деления

In [112]:
R.<x,y> = PolynomialRing(QQ, order='lex')

g = x^7*y^2 + x^3*y^2 - y +  1
f1 = x*y^2  - x 
f2 = x - y^3

q, r = divide_with_rem(g, [f1, f2], verbose=True)

after step 0 r=x^7*y^2 + x^3*y^2 - y + 1,  q=[0, 0], rem=0:
after step 1 r=x^7 + x^3*y^2 - y + 1,  q=[x^6, 0], rem=0:
after step 2 r=x^6*y^3 + x^3*y^2 - y + 1,  q=[x^6, x^6], rem=0:
after step 3 r=x^6*y + x^3*y^2 - y + 1,  q=[x^6 + x^5*y, x^6], rem=0:
after step 4 r=x^5*y^4 + x^3*y^2 - y + 1,  q=[x^6 + x^5*y, x^6 + x^5*y], rem=0:
after step 5 r=x^5*y^2 + x^3*y^2 - y + 1,  q=[x^6 + x^5*y + x^4*y^2, x^6 + x^5*y], rem=0:
after step 6 r=x^5 + x^3*y^2 - y + 1,  q=[x^6 + x^5*y + x^4*y^2 + x^4, x^6 + x^5*y], rem=0:
after step 7 r=x^4*y^3 + x^3*y^2 - y + 1,  q=[x^6 + x^5*y + x^4*y^2 + x^4, x^6 + x^5*y + x^4], rem=0:
after step 8 r=x^4*y + x^3*y^2 - y + 1,  q=[x^6 + x^5*y + x^4*y^2 + x^4 + x^3*y, x^6 + x^5*y + x^4], rem=0:
after step 9 r=x^3*y^4 + x^3*y^2 - y + 1,  q=[x^6 + x^5*y + x^4*y^2 + x^4 + x^3*y, x^6 + x^5*y + x^4 + x^3*y], rem=0:
after step 10 r=2*x^3*y^2 - y + 1,  q=[x^6 + x^5*y + x^4*y^2 + x^4 + x^3*y + x^2*y^2, x^6 + x^5*y + x^4 + x^3*y], rem=0:
after step 11 r=2*x^3 - y + 1,  q=[x^

Поменяем порядок переменных

In [113]:
R.<y,x> = PolynomialRing(QQ, order='lex')

g = x^7*y^2 + x^3*y^2 - y +  1
f1 = x*y^2  - x 
f2 = x - y^3

q, r = divide_with_rem(g, [f1, f2], verbose=True)

after step 0 r=y^2*x^7 + y^2*x^3 - y + 1,  q=[0, 0], rem=0:
after step 1 r=y^2*x^3 - y + x^7 + 1,  q=[x^6, 0], rem=0:
after step 2 r=-y + x^7 + x^3 + 1,  q=[x^6 + x^2, 0], rem=0:
after step 3 r=x^7 + x^3 + 1,  q=[x^6 + x^2, 0], rem=-y:
after step 4 r=x^3 + 1,  q=[x^6 + x^2, 0], rem=-y + x^7:
after step 5 r=1,  q=[x^6 + x^2, 0], rem=-y + x^7 + x^3:
after step 6 r=0,  q=[x^6 + x^2, 0], rem=-y + x^7 + x^3 + 1:


Поменяем мономиальный порядок

In [114]:
R.<x,y> = PolynomialRing(QQ, order='deglex')

g = x^7*y^2 + x^3*y^2 - y +  1
f1 = x*y^2  - x 
f2 = x - y^3

q, r = divide_with_rem(g, [f1, f2], verbose=True)

after step 0 r=x^7*y^2 + x^3*y^2 - y + 1,  q=[0, 0], rem=0:
after step 1 r=x^7 + x^3*y^2 - y + 1,  q=[x^6, 0], rem=0:
after step 2 r=x^3*y^2 - y + 1,  q=[x^6, 0], rem=x^7:
after step 3 r=x^3 - y + 1,  q=[x^6 + x^2, 0], rem=x^7:
after step 4 r=-y + 1,  q=[x^6 + x^2, 0], rem=x^7 + x^3:
after step 5 r=1,  q=[x^6 + x^2, 0], rem=x^7 + x^3 - y:
after step 6 r=0,  q=[x^6 + x^2, 0], rem=x^7 + x^3 - y + 1:


Рассмотрим пример, при котором частные разные, но остатки равны

In [102]:
R.<x,y, z> = PolynomialRing(QQ, order='deglex')

g = x*y 
f1 = x+z 
f2 = y-z

q, r = divide_with_rem(g, [f2, f1], verbose=True)
print()
q, r = divide_with_rem(g, [f1, f2], verbose=True)

after step 0 r=x*y,  q=[0, 0], rem=0:
after step 1 r=x*z,  q=[x, 0], rem=0:
after step 2 r=-z^2,  q=[x, z], rem=0:
after step 3 r=0,  q=[x, z], rem=-z^2:

after step 0 r=x*y,  q=[0, 0], rem=0:
after step 1 r=-y*z,  q=[y, 0], rem=0:
after step 2 r=-z^2,  q=[y, -z], rem=0:
after step 3 r=0,  q=[y, -z], rem=-z^2:


Протестируйте свой код

In [119]:
## your code here

Ответьте на вопрос, как генерировать примеры многочленов, таких что остаток от деления не зависит от порядка многочленов на которые мы делим.

In [ ]:
## your code here

### Построение базисов Гребнера

Сналала напишем функцию вычисления S-многочленов. Для вычисления наибольшего общего кратного будем использовать встроенную фнукицю `lcm`

In [115]:
R.<x,y, z> = PolynomialRing(QQ, order='lex')

f = x^3*y^2
g = x^5*y*z
print(lcm(f,g))

x^5*y^2*z


In [103]:
def syz(f, g):
    num = lcm(f.lm(), g.lm())
    fir = (num / f.lt()) * f
    sec = # your code here
    ans = fir - sec
    return f.parent()(ans)

Используя алгоритм  Бухбергера, напишем базовый алгоритм вычисления базиса Гребнера, базирующийся на добавлении ненулевых остатков S-пар. 

In [ ]:
def compute_groebner(F, verbose = False):
    ans = [f for f in F]
    
    is_finished = False
    idx_iteration = 0
    while not is_finished:
        
        if verbose:
            print(f'after {idx_iteration} iteration')
            for f in ans:
                print(f)
            print()
        
        n = len(ans)
        is_finished = True
        for i in range(n):
            for j in range(i+1, n):
                s = syz(ans[i],ans[j])
                _, r = divide_with_rem(s, # your code here)
                if not r.is_zero():
                    ans.append(r)
                    is_finished = False
        
        idx_iteration += 1
    return ans

Отметим, что в приводимой ниже версии алгоритма происходят лишние вычисления. Какие?

In [ ]:
# your code here

Проверим работоспособность нашего кода

In [116]:
compute_groebner(F = [x^3 - 2*x*y, x^2*y - 2*y^2 + x], verbose = True)

after 0 iteration
x^3 - 2*x*y
x^2*y + x - 2*y^2

after 1 iteration
x^3 - 2*x*y
x^2*y + x - 2*y^2
-x^2

after 2 iteration
x^3 - 2*x*y
x^2*y + x - 2*y^2
-x^2
-2*x*y
x - 2*y^2

after 3 iteration
x^3 - 2*x*y
x^2*y + x - 2*y^2
-x^2
-2*x*y
x - 2*y^2
4*y^3



[x^3 - 2*x*y, x^2*y + x - 2*y^2, -x^2, -2*x*y, x - 2*y^2, 4*y^3]

Протестируйте свой код

In [120]:
## your code here

Далее напишем вспомогательную функцию проверки того, что множество многочленов является базисом Гребнера

In [105]:
def is_groebner(F):
    n = len(F)
    gb = compute_groebner(F, verbose=False)
    return n == len(gb)

### Минимальный базис Гребнера

Далее напишем функцию для вычисления минимального базиса Гребнера, базирущуюся на идее удааления из некоторого базиса Гребнера многочленов со старшим членом, делящимся на старший член некоторого другого многочлена

In [117]:
def compute_minimal_groebner(F, verbose=False):
    gb = compute_groebner(F, verbose)
    n = len(gb)
    pick_idxes = [True] * n
    for i in range(n):
        for j in range(n):
            if i == j:
                continue
            if gb[j].lm().divides(gb[i].lm()):
                pick_idxes[i] = # your code here
                break
    ans = []
    for i in range(n):
        if pick_idxes[i]:
            ans.append(gb[i])
    return ans

Проверим наш метод

In [118]:
compute_minimal_groebner(F = [x^3 - 2*x*y, x^2*y - 2*y^2 + x], verbose = True)

after 0 iteration
x^3 - 2*x*y
x^2*y + x - 2*y^2

after 1 iteration
x^3 - 2*x*y
x^2*y + x - 2*y^2
-x^2

after 2 iteration
x^3 - 2*x*y
x^2*y + x - 2*y^2
-x^2
-2*x*y
x - 2*y^2

after 3 iteration
x^3 - 2*x*y
x^2*y + x - 2*y^2
-x^2
-2*x*y
x - 2*y^2
4*y^3



[x - 2*y^2, 4*y^3]

Протестируйте свой код

In [ ]:
## your code here

### Редуцированный базис Гребнера

Ниже мы напишем функцию для вычисления редуцированного базиса Гребнера. Обратите внимание, что в редуцированный базис мы добавляем только унитарные многочлены

In [ ]:
def compute_reduced_groebner(F, verbose=False):
    mgb = compute_minimal_groebner(F, verbose)
    n = len(mgb)
    
    ans = []
    for i in range(n):
        reducing_pols = ans + mgb[i+1:]
        _, g = divide_with_rem(mgb[i], reducing_pols, verbose)
        ans.append(g / g.lc())
    return ans

Проверим работоспособность кода

In [121]:
compute_reduced_groebner(F = [x^3 - 2*x*y, x^2*y - 2*y^2 + x], verbose = True)

after 0 iteration
x^3 - 2*x*y
x^2*y + x - 2*y^2

after 1 iteration
x^3 - 2*x*y
x^2*y + x - 2*y^2
-x^2

after 2 iteration
x^3 - 2*x*y
x^2*y + x - 2*y^2
-x^2
-2*x*y
x - 2*y^2

after 3 iteration
x^3 - 2*x*y
x^2*y + x - 2*y^2
-x^2
-2*x*y
x - 2*y^2
4*y^3

after step 0 r=x - 2*y^2,  q=[0], rem=0:
after step 1 r=-2*y^2,  q=[0], rem=x:
after step 2 r=0,  q=[0], rem=x - 2*y^2:
after step 0 r=4*y^3,  q=[0], rem=0:
after step 1 r=0,  q=[0], rem=4*y^3:


[x - 2*y^2, y^3]

In [109]:
R.<x,y, z> = PolynomialRing(QQ, order='lex')

compute_reduced_groebner(F = [x^2 + y^2 + z^2 - 1, x^2 + z^2 - y, x-z], verbose = True)

after 0 iteration
x^2 + y^2 + z^2 - 1
x^2 - y + z^2
x - z

after 1 iteration
x^2 + y^2 + z^2 - 1
x^2 - y + z^2
x - z
y^2 + y - 1
-y + 2*z^2

after 2 iteration
x^2 + y^2 + z^2 - 1
x^2 - y + z^2
x - z
y^2 + y - 1
-y + 2*z^2
4*z^4 + 2*z^2 - 1

after step 0 r=x - z,  q=[0, 0], rem=0:
after step 1 r=-z,  q=[0, 0], rem=x:
after step 2 r=0,  q=[0, 0], rem=x - z:
after step 0 r=-y + 2*z^2,  q=[0, 0], rem=0:
after step 1 r=2*z^2,  q=[0, 0], rem=-y:
after step 2 r=0,  q=[0, 0], rem=-y + 2*z^2:
after step 0 r=4*z^4 + 2*z^2 - 1,  q=[0, 0], rem=0:
after step 1 r=2*z^2 - 1,  q=[0, 0], rem=4*z^4:
after step 2 r=-1,  q=[0, 0], rem=4*z^4 + 2*z^2:
after step 3 r=0,  q=[0, 0], rem=4*z^4 + 2*z^2 - 1:


[x - z, y - 2*z^2, z^4 + 1/2*z^2 - 1/4]

Проверим себя, вспомнив, что Sage из коробки считает редуцированный базис Гребнера

In [125]:
R.<x,y, z> = PolynomialRing(QQ, order='lex')

f = x^2 + y^2 + z^2 - 1
g = x^2 + z^2 - y
h = x-z

I = Ideal([f, g, h])
rgb = I.groebner_basis()
rgb

[x - z, y - 2*z^2, z^4 + 1/2*z^2 - 1/4]

Видим, что результаты совпадают

Напишите тесты

In [126]:
## your code here

Напишите выводы о проделанной работе

In [ ]:
## your code here